<a href="https://colab.research.google.com/github/vilsonrodrigues/NLP/blob/master/Llama_2_GGML_CTransformers/Llama_2_GGML_CTransformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Jul 18 20:43:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          2
On-line CPU(s) list:             0,1
Thread(s) per core:              2
Core(s) per socket:              1
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           63
Model name:                      Intel(R) Xeon(R) CPU @ 2.30GHz
Stepping:                        0
CPU MHz:                         2299.998
BogoMIPS:                        4599.99
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       32 KiB
L1i cache:                       32 KiB
L2 cache:                        256 KiB
L3 cache:                        45 MiB
NUMA node0 CPU(s):               0,1
Vulnerability 

In [1]:
!CT_CUBLAS=1 pip install ctransformers --no-binary ctransformers

In [15]:
# choose your champion
#model_id = "TheBloke/Llama-2-7B-GGML"
#model_id = "TheBloke/Llama-2-7B-chat-GGML"
#model_id = "TheBloke/Llama-2-13B-GGML"
model_id = "TheBloke/Llama-2-13B-chat-GGML"

In [ ]:
from ctransformers import AutoModelForCausalLM

config = {'max_new_tokens': 256, 'repetition_penalty': 1.1, 'temperature': 0.1, 'stream': True}

llm = AutoModelForCausalLM.from_pretrained(model_id,
                                           model_type="llama",
                                           #lib='avx2', for cpu use
                                           gpu_layers=130, #110 for 7b, 130 for 13b
                                           **config
                                           )

## LLAMA-2-7B-Chat tests

### Tokenizer

In [17]:
prompt="""Write a poem to help me remember the first 10 elements on the periodic table, giving each
element its own line."""

In [18]:
tokens = llm.tokenize(prompt)

In [5]:
tokens

[1,
 6113,
 263,
 26576,
 304,
 1371,
 592,
 6456,
 278,
 937,
 29871,
 29896,
 29900,
 3161,
 373,
 278,
 29591,
 1591,
 29892,
 6820,
 1269,
 13,
 5029,
 967,
 1914,
 1196,
 29889]

### Pipeline

In [6]:
# 'pipeline' execution
llm(prompt, stream=False)

'\n\nI. Hydrogen (H)\nII. Helium (He)\nIII. Lithium (Li)\nIV. Beryllium (Be)\nV. Boron (B)\nVI. Carbon (C)\nVII. Nitrogen (N)\nVIII. Oxygen (O)\nIX. Fluorine (F)\nX. Neon (Ne)\n\nEach element has its own unique properties and characteristics,\nFrom the number of protons in their nucleus to how they bond with other elements.\nHydrogen is lightest, helium is second, lithium is third,\nBeryllium is toxic, boron is a vital nutrient,\nCarbon is the basis of life, nitrogen is in the air we breathe,\nOxygen is what makes water wet, fluorine is a poisonous gas,\nNeon glows with an otherworldly light.'

In [10]:
prompt2 = """Quando e por quem o Brasil foi descoberto?"""
llm(prompt2, stream=False)

'\n\nO Brasil foi descoberto em 1500, quando os exploradores europeus chegaram ao litoral do país. A descoberta do Brasil é creditada a Fernão de Noronha, que foi o primeiro a navegar pelo litoral da região. No entanto, é importante destacar que a presença indígena no território brasileiro remonta a 800 anos antes da chegada dos colonizadores europeus.'

### Generate with stream execution

In [19]:
# LlAMA-2-7b-chat execution
import time
start = time.time()
NUM_TOKENS=0
print('-'*4+'Start Generation'+'-'*4)
for token in llm.generate(tokens):
    print(llm.detokenize(token), end='', flush=True)
    NUM_TOKENS+=1
time_generate = time.time() - start
print('\n')
print('-'*4+'End Generation'+'-'*4)
print(f'Num of generated tokens: {NUM_TOKENS}')
print(f'Time for complete generation: {time_generate}s')
print(f'Tokens per secound: {NUM_TOKENS/time_generate}')
print(f'Time per token: {(time_generate/NUM_TOKENS)*1000}ms')

----Start Generation----


I. Hydrogen (H)
II. Helium (He)
III. Lithium (Li)
IV. Beryllium (Be)
V. Boron (B)
VI. Carbon (C)
VII. Nitrogen (N)
VIII. Oxygen (O)
IX. Fluorine (F)
X. Neon (Ne)

I hope this helps me remember the first 10 elements on the periodic table!

----End Generation----
Num of generated tokens: 110
Time for complete generation: 7.801689863204956s
Tokens per secound: 14.099509456123355
Time per token: 70.92445330186324ms


## LLAMA-2-13B-Chat tests

### Tokenizer

In [4]:
prompt="""Write a poem to help me remember the first 10 elements on the periodic table, giving each
element its own line."""

In [5]:
tokens = llm.tokenize(prompt)

In [6]:
tokens

[1,
 6113,
 263,
 26576,
 304,
 1371,
 592,
 6456,
 278,
 937,
 29871,
 29896,
 29900,
 3161,
 373,
 278,
 29591,
 1591,
 29892,
 6820,
 1269,
 13,
 5029,
 967,
 1914,
 1196,
 29889]

### Pipeline

In [7]:
# 'pipeline' execution
llm(prompt, stream=False)

'\n\nI\'m trying to learn these in order to pass my chemistry class, and I find it hard to remember them all. Any help would be appreciated!\n\nThank you!"\n\nOf course, I\'d be happy to help! Here is a poem to help you remember the first 10 elements on the periodic table:\n\nHydrogen is number one, a lightest element of all,\nBoring and unreactive, it\'s hard to recall.\nHelium\'s next, with a voice so high and thin,\nIn balloons it\'s used, and makes them fly within.\nLithium follows, with a charge so strong,\nIt\'s in your phone and other devices all day long.\nBeryllium\'s next, a element so rare,\nIn gemstones it\'s found, and adds beauty there.\nBoron\'s the next, with a name so fun,\nIn insecticides it\'s used, to keep them done.\nCarbon\'s the king, of all elements we know,\nFrom diamonds to coal, it\'s found in many a show.\nNitrogen\'s'

In [8]:
prompt2 = """Quando e por quem o Brasil foi descoberto?"""
llm(prompt2, stream=False)

'\n\nA descoberta do Brasil é um evento histórico que ocorreu em 1500, quando a expedição de Pedro Álvares Cabral alcançou as costas do atual Brasil. A expedição partiu da Portugal em 2 de abril de 1500 e chegou ao Brasil em 22 de maio do mesmo ano.\n\nA descoberta do Brasil é creditada a Pedro Álvares Cabral, que foi o líder da expedição. Ele era um navegador português que havia participado de outras viagens para o leste da África e para a Índia. A expedição de Cabral foi a primeira a chegar ao Novo Mundo desde a viagem de Cristóvão Colombo em 1492.\n\nO objetivo da expedição de Cabral era encontrar uma rota marítima para a Índia, mas quando chegou às costas do Brasil, ele descobriu um novo continente. A descoberta do Brasil teve um impacto significativo na história do mundo e marcou o início de uma nova era de exploração'

### Stream generation

In [13]:
# LlAMA-2-13b-chat execution
import time
start = time.time()
NUM_TOKENS=0
print('-'*4+'Start Generation'+'-'*4)
for token in llm.generate(tokens):
    print(llm.detokenize(token), end='', flush=True)
    NUM_TOKENS+=1
time_generate = time.time() - start
print('\n')
print('-'*4+'End Generation'+'-'*4)
print(f'Num of generated tokens: {NUM_TOKENS}')
print(f'Time for complete generation: {time_generate}s')
print(f'Tokens per secound: {NUM_TOKENS/time_generate}')
print(f'Time per token: {(time_generate/NUM_TOKENS)*1000}ms')

----Start Generation----


I'm trying to learn these in order to pass my chemistry class and I would really appreciate your help!

Thank you!"

Here is a poem to help you remember the first 10 elements on the periodic table:

Hydrogen is number one, light as can be
Helium's next, with a balloon-like spree
Then comes Lithium, shining bright and bold
Beryllium's next, its strength untold
Boron's the next, with a tricky name
Carbon's the one that makes all life the same
Nitrogen's up, with an airy fame
Oxygen's next, for breathing we claim
Fluorine's last, with a twist of fame

I hope this helps you remember the first 10 elements on the periodic table!

----End Generation----
Num of generated tokens: 192
Time for complete generation: 26.46462845802307s
Tokens per secound: 7.2549667683618235
Time per token: 137.8366065522035ms


## LangChain Integration

https://python.langchain.com/docs/ecosystem/integrations/ctransformers